# Field Goals

In [ ]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt

In [ ]:
play_types = {"field_goal_good": 59,
              "field_goal_missed": 60,
              "field_goal_blocked": 18,
              "field_goal_missed_return": 40,
              "field_goal_missed_return_td": 41,
              "field_goal_blocked_return_td": 38}

In [ ]:
seasons = np.arange(2000, 2020, 1) 
weeks = np.arange(1, 14, 1)

In [ ]:
season_list = []
week_list = []
kicks_json = []

for season in seasons:
    for week in weeks:
        for key in play_types:
            parameters = {"year": season, "seasonType": "regular", "offense": "Washington", "playType": play_types[key], "week": week}
            response = requests.get("https://api.collegefootballdata.com/plays", params=parameters)
            kicks = response.json()
            for kick in kicks:
                season_list.append(season)
                week_list.append(week)
                kicks_json.append(kick)

In [ ]:
from pandas.io.json import json_normalize

kick_df = json_normalize(kicks_json)
kick_df["season"] = season_list
kick_df["week"] = week_list
kick_df.head()

In [ ]:
kick_df.shape

In [ ]:
kick_df.play_type.unique()

In [ ]:
colors = {"Blocked Field Goal": "black",
          "Field Goal Good": "green",
          "Field Goal Missed": "red"}

In [ ]:
kick_df["color"] = kick_df["play_type"].map(colors)

In [ ]:
plt.scatter(kick_df.year, kick_df.distance, s = size_map, c=kick_df.color)
plt.show()